In [1]:
# Import necessary libraries
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import keras_tuner as kt

2024-10-02 13:31:20.570688: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-02 13:31:20.811874: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-02 13:31:20.818085: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-02 13:31:21.036147: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-02 13:31:23.333884: W tensorflow/compiler/tf2

In [2]:
# Paths
data_dir = 'data/training'  # Folder with .tif images
csv_file = 'data/training.csv'  # CSV file with image_id and is_homogeneous
IMG_SIZE = (224, 224)
BATCH_SIZE = 8

# Load the CSV file
df = pd.read_csv(csv_file)

# Ensure correct column names
df.columns = df.columns.str.strip()
df.rename(columns={'is_homogenous': 'is_homogeneous'}, inplace=True)

# Create lists of image paths and labels
image_paths = [os.path.join(data_dir, f"{str(image_id).zfill(3)}.tif") for image_id in df['image_id']]
labels = df['is_homogeneous'].values

def load_and_preprocess_image(image_path):
    img = load_img(image_path, target_size=IMG_SIZE)
    img_array = img_to_array(img)
    img_array = img_array / 255.0  # Normalize pixel values
    return img_array

# Load images
images = np.array([load_and_preprocess_image(image_path) for image_path in image_paths])

# Stratify train-test split
X_train, X_val, y_train, y_val = train_test_split(
    images, labels, test_size=0.2, random_state=42, stratify=labels
)


In [3]:
def build_model(hp):
    # Load the pre-trained ResNet50 model without the top layer
    base_model = ResNet50(
        weights='imagenet', include_top=False, input_shape=(224, 224, 3)
    )
    
    # Freeze all layers initially
    for layer in base_model.layers:
        layer.trainable = False
    
    # Option to unfreeze last n layers
    unfreeze_layers = hp.Int('unfreeze_layers', min_value=0, max_value=10, step=1)
    if unfreeze_layers > 0:
        for layer in base_model.layers[-unfreeze_layers:]:
            layer.trainable = True
    
    # Add custom layers on top of the base model
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    
    # Tune the number of units and dropout rate
    units = hp.Int('units', min_value=64, max_value=256, step=64)
    x = Dense(units, activation='relu')(x)
    
    dropout_rate = hp.Float('dropout_rate', min_value=0.3, max_value=0.7, step=0.1)
    x = Dropout(dropout_rate)(x)
    
    # Second Dense layer
    units2 = hp.Int('units2', min_value=32, max_value=128, step=32)
    x = Dense(units2, activation='relu')(x)
    
    # Output layer
    predictions = Dense(1, activation='sigmoid')(x)
    
    # Create the model
    model = Model(inputs=base_model.input, outputs=predictions)
    
    # Tune the learning rate
    learning_rate = hp.Float('learning_rate', min_value=1e-5, max_value=1e-3, sampling='LOG')
    
    # Compile the model
    model.compile(
        optimizer=Adam(learning_rate=learning_rate),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    
    return model


In [5]:
class MyTuner(kt.engine.tuner.Tuner):
    def __init__(self, X_train, y_train, X_val, y_val, **kwargs):
        super(MyTuner, self).__init__(**kwargs)
        self.X_train = X_train
        self.y_train = y_train
        self.X_val = X_val
        self.y_val = y_val

    def run_trial(self, trial, **fit_kwargs):
        hp = trial.hyperparameters
        
        # Create data augmentation parameters using hp
        train_datagen = ImageDataGenerator(
            rotation_range=hp.Int('rotation_range', 0, 40, step=10),
            zoom_range=hp.Float('zoom_range', 0.0, 0.3, step=0.1),
            horizontal_flip=True,
            vertical_flip=True,
            fill_mode='nearest'
        )
        
        # No augmentation for validation data
        val_datagen = ImageDataGenerator()
        
        # Create data generators
        train_generator = train_datagen.flow(
            self.X_train, self.y_train, batch_size=BATCH_SIZE
        )
        val_generator = val_datagen.flow(
            self.X_val, self.y_val, batch_size=BATCH_SIZE
        )
        
        # Build model
        model = self.hypermodel.build(hp)
        
        # Get class weights
        class_weight_0 = hp.Float('class_weight_0', 0.5, 2.0, step=0.5)
        class_weight_1 = hp.Float('class_weight_1', 1.0, 5.0, step=1.0)
        class_weights = {0: class_weight_0, 1: class_weight_1}
        
        # Add class_weights to fit_kwargs
        fit_kwargs['class_weight'] = class_weights
        
        # Add callbacks
        callbacks = [
            EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
            ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)
        ]
        fit_kwargs['callbacks'] = callbacks
        
        # Run trial
        return super(MyTuner, self).run_trial(
            trial, train_generator, validation_data=val_generator, **fit_kwargs
        )

In [6]:
tuner = MyTuner(
    hypermodel=build_model,
    objective=kt.Objective('val_accuracy', direction='max'),
    max_trials=20,
    executions_per_trial=1,
    directory='hyperparam_tuning',
    project_name='resnet50_tuning',
    X_train=X_train,
    y_train=y_train,
    X_val=X_val,
    y_val=y_val
)

TypeError: __init__() missing 1 required positional argument: 'oracle'